# Worksheet 04

Name: Yingjie Ouyang  
UID: U86074230

### Topics

- Distance & Similarity
- Cost Functions
- K means

### Distance & Similarity

#### Part 1

a) In the minkowski distance, describe what the parameters p and d are.

p refers to a parameter that determines flexibility of distance  
d refers to the dimension in real space we are operating in

b) In your own words describe the difference between the Euclidean distance and the Manhattan distance.

Euclidean distance is the shortest distance between two points i.e. if you drew a line between the two points

Manhattan distance is the shortest distance between two points along the x/y axis i.e. if you could only move horizontally or vertically

Consider A = (0, 0) and B = (1, 1). When:

- p = 1, d(A, B) = 2
- p = 2, d(A, B) = $\sqrt{2} = 1.41$
- p = 3, d(A, B) = $2^{1/3} = 1.26$
- p = 4, d(A, B) = $2^{1/4} = 1.19$

c) Describe what you think distance would look like when p is very large.

The distance between A and B starts to converge at 1

d) Is the minkowski distance still a distance function when p < 1? Expain why / why not.

No, because it violates the triangle inequality

e) when would you use cosine similarity over the euclidan distance?

When direction matters more than magnitude

f) what does the jaccard distance account for that the manhattan distance doesn't?

The size of the intersection

#### Part 2

Consider the following two sentences:

In [1]:
s1 = "hello my name is Alice"  
s2 = "hello my name is Bob"

using the union of words from both sentences, we can represent each sentence as a vector. Each element of the vector represents the presence or absence of the word at that index.

In this example, the union of words is ("hello", "my", "name", "is", "Alice", "Bob") so we can represent the above sentences as such:

In [2]:
v1 = [1,    1, 1,   1, 1,    0]
#     hello my name is Alice
v2 = [1,    1, 1,   1, 0, 1]
#     hello my name is    Bob

Programmatically, we can do the following:

In [3]:
corpus = [s1, s2]
all_words = list(set([item for x in corpus for item in x.split()]))
print(all_words)
v1 = [1 if x in s1 else 0 for x in all_words]
print(v1)

['hello', 'is', 'Alice', 'Bob', 'my', 'name']
[1, 1, 1, 0, 1, 1]


Let's add a new sentence to our corpus:

In [4]:
s3 = "hi my name is Claude"
corpus.append(s3)

a) What is the new union of words used to represent s1, s2, and s3?

In [5]:
all_words = list(set([item for x in corpus for item in x.split()]))
print(all_words)

['hello', 'hi', 'is', 'Alice', 'Bob', 'my', 'Claude', 'name']


b) Represent s1, s2, and s3 as vectors as above, using this new set of words.

In [6]:
v1 = [1 if x in s1 else 0 for x in all_words]
print(v1)
v2 = [1 if x in s2 else 0 for x in all_words]
print(v2)
v3 = [1 if x in s3 else 0 for x in all_words]
print(v3)

[1, 0, 1, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 1, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 1]


c) Write a function that computes the manhattan distance between two vectors. Which pair of vectors are the most similar under that distance function?

In [9]:
def manhattan_dist(x, y):
  # TODO: check that x and y are of the same dimension
  res = 0
  for i in range (len(x)):
    res += abs(x[i] - y[i])
  return res

print(manhattan_dist(v1, v2))
print(manhattan_dist(v2, v3))
print(manhattan_dist(v3, v1))

2
4
4


v1 and v2 are the most similar under manhattan distance

d) Create a matrix of all these vectors (row major) and add the following sentences in vector form:

- "hi Alice"
- "hello Claude"
- "Bob my name is Claude"
- "hi Claude my name is Alice"
- "hello Bob"

In [13]:
sentences = ["hi Alice", "hello Claude", "Bob my name is Claude", "hi Claude my name is Alice", "hello Bob"]
all_words = list(set([item for x in sentences for item in x.split()]))
print(all_words)
matrix = [[1 if x in s else 0 for x in all_words] for s in sentences]
print(matrix)

['hello', 'hi', 'is', 'Alice', 'Bob', 'my', 'Claude', 'name']
[[0, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 1], [1, 0, 0, 0, 1, 0, 0, 0]]


e) How many rows and columns does this matrix have?

In [15]:
print("(", len(matrix), "x", len(matrix[0]), ")")

( 5 x 8 )


f) When using the Manhattan distance, which two sentences are the most similar?

In [18]:
for i in range(len(matrix)):
  for j in range(i+1, len(matrix)):
    print("Manhattan distance for", sentences[i], "and", sentences[j])
    print(manhattan_dist(matrix[i], matrix[j]))

Manhattan distance for hi Alice and hello Claude
4
Manhattan distance for hi Alice and Bob my name is Claude
7
Manhattan distance for hi Alice and hi Claude my name is Alice
4
Manhattan distance for hi Alice and hello Bob
4
Manhattan distance for hello Claude and Bob my name is Claude
5
Manhattan distance for hello Claude and hi Claude my name is Alice
6
Manhattan distance for hello Claude and hello Bob
2
Manhattan distance for Bob my name is Claude and hi Claude my name is Alice
3
Manhattan distance for Bob my name is Claude and hello Bob
5
Manhattan distance for hi Claude my name is Alice and hello Bob
8


"hello Claude" and "hello Bob"

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

For center 0, we have [0, 0.5]  
For center 2 we have [1.5, 2, 6, 6.5, 7]

The new centers are the mean of center 0 = (0 + 0.5) / 2 = 0.25  
and the mean of center 2 = (1.5 + 2 + 6 + 6.5 + 7) / 5 = 4.6

For center 0.25 we have [0, 0.5, 1.5, 2]  
For center 4.6 we have [6, 6.5, 7]

The new centers are (0 + 0.5 + 1.5 + 2) / 4 = 1  
and (6 + 6.5 + 7) / 3 = 6.5

For center 1 we have [0, 0.5, 1.5, 2]  
For center 6.5 we have [6, 6.5, 7]  
Since the values haven't moved to any other cluster, we have converged.

b) Describe in plain english what the cost function for k means is.

Want to find k points such that the overall distance between the centers and their assigned data points are minimized

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

Depending on where the centers are, each cluster may have different data points

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, because it's a greedy algorithm so it always attempts to find the most optimal solution per iteration. However, it does not guarantee it will be the most optimal solution, as the algorithm could converge to a local optima.